First LLM models relied entirely on their own knowledge. Problem: the knowledge of first ChatGPT was 2 years behind the actual time. To overcome this you had to continuosly retrain the whole model which is unfeasible.

Classic RAG system:
1. Retriever component fetches candidate documents (focus on recall)<br>
   - using query matching
   - using embedding matching
2. LLM uses them as a context

Challenge, not only in the domain of LLM generation but in the IR generally = original user query might be not enough for retireval of all relevant documents

__User query insufficiency (1982)__<br>
[[paper](https://arxiv.org/pdf/2503.00223)]<br> Users often cannot articulate their needs precisely because they don’t fully understand the problem => IR systems must be context-dependent, personalized, dialogue maintaining

__Rocchio (1971)__<br>
Rocchio algorithm defined the first personalized version of document Retrieval. What it does - it shifts query vector (from the document-query vector space) in the direction towards documents previously positively evaluated by this user

__Query Augmentation__ = rewriting user query to reflect all sides of user's intent

__PRF (2021)__<br>[[paper]](https://arxiv.org/abs/2108.11044)<br>PRF = Pseudo-Relevance Feedback. It is the algorithm to enhance the original query with related / synonim terms. Synonymic terms = most frequent terms that appear in the list of documents returned on the first retrieval

<img src="img/prf.png" width=400>

__ColBERT (2020)__<br>
In late 2010s transformer-based Encoders (BERT) narrowed the gap by learning to extract the latent semantics of the query and documents. Late linkage models like ColBERT made the process efficient

__Zero-shot retrieval__<br>
Out-of-the box pretrained Encoders do not know how to rank by relevance. They need to be fine-tuned on some relevance dataset. Usually contrastive losses are used. But labeling is expensive => there is a challenge of __zero-shot retrieval__ - fetching without previous training on human labeled data

__DPR-CTL (2020)__<br>A neural retrieval method that is trained in self-supervised fashion - neighboring chunks of text are considered positive exmaples, random chunks - negative ones. It achieves comparable to fine-tuned alternatives performance 

__Dense Retrieval__<br>Dense Retrieval is a family of algorithms where queries and documents are mapped into the same latent vector-space by some sort of Encoder. Query/document proximity is then calculated by a cosine / dot-product similarity

__DPR (2020)__<br>[[paper]](https://arxiv.org/pdf/2004.04906)<br>DPR = Dense Passage Model is an implementation of Dense Retrieval offered by Meta. It uses two BERT models to encode query and documents and dot-product as similarity measure. Encoders are trained over contrastive loss (one positive and one negative example)

__RAG (2020)__<br>[[paper]](https://arxiv.org/pdf/2005.11401)<br>Retrieval Augmented Generation = the first implementation of the RAG approach when the term was coined. It uses DPR retrieval combined with some generation model (BART in the original paper)

<img src="img/rag.png" width=600>

__ExaRanker (2023)__<br>
[[paper]](https://arxiv.org/pdf/2402.06334)<br>
Before training the Retriever on some relevance dataset let's use a strong LLM (like Chatgpt) to generate a textual "explanation" for each example in this dataset ("this document is relevant because ...")

During training phase make the Retriever model not only predict the correct label, but also reconstruct this explanation<br>This develops model's reasoning ability about relevance, avoids prediction hacking. Distance to ground-truth explanation is measured using standard text (sequence-to-sequence) loss

<img src="img/exarank.png" width=400>

__CONQRR (2022)__<br>
[[paper]](https://arxiv.org/pdf/2112.08558)<br>
Retrieval for conversational (dialog) systems is more challenging since the query might be distributed along the previous conversation ("What about his birthplace?"). CONQRR summarizes all necessary information for the query to be effectively processed. It is retriever agnostic and relies only on query rewriting
<img src="img/conqrr.png" width=400>

__Contextual Clues Sampling (2022)__<br>
[[paper]](https://arxiv.org/pdf/2210.07093)<br>
The authors suggest using some strong LLM (ChatGPT) to enhance original query by generating a list of related terms - they call them "contextual clues"<br>Not sure what exact prompt do they use "Model, generate me related terms"?

Retriever model runs multiple fetches - one for each enhancement and extracts a list of documents which are next fused into one large list.

Diversity is achieved by multiple generations. They are followed by deduplication - identic or similar "clues" are grouped into clusters<br>Precision is achieved by first ranking the clues and then retrieved documents according to their generation probabilities. Only top-K are used.

<img src="img/contextual_clues.png" width=400>

__DeepRetrieval (2025)__<br>[[paper]](https://arxiv.org/pdf/2503.00223)<br>uses a separate reasoning LLM model to reformulate user query. Generator is not mentioned

The model is updated using RL (PPO). Reward consists of output query consistency (how good the prompt is formatted) + Recall-based reward (how much relevant documents we fetched)

__Search-R1 (2025)__<br>[[paper]](https://arxiv.org/pdf/2503.09516)<br>This model incorporates searching calls in the reasoning process and makes it iterative (retrieve + generate). No split into searcher + generator. The model is updated using DPO/GRPO. Reward is determined by the correctness of the final answer (ExactMatch)

__S3__<br>
We don't always have access to the generator LLM weights (it could be ChatGPT) => let's differentiate between Search & Generator and fine-tune the Retrieval component

In s3 (Search, select, serve) we train the Retrieval model using RL. We use final answer as a reward but target in the uplift compared to the baseline retrieval